CONNECTION

In [34]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json
import os

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        try:
            db = create_engine('postgresql+psycopg2://'+db_user+':'+db_pw+'@'+host+'/'+default_db, echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(sqlcmd, args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

db, conn = pgconnect(credentials)

Connected successfully.


LOADING OF DATA + CLEANING FOR TABLES BUSINESSES, POPULATION, INCOME, POLLING, STOPS AND CATCHMENTS

BUSINESSES

In [10]:


conn.execute('DROP TABLE IF EXISTS BUSINESSES')

#LOAD DATA
import pandas as pd
data_businesses = pd.read_csv('Businesses.csv')

conn.execute("""
CREATE TABLE BUSINESSES (
industry_name VARCHAR(50),
sa2_code INT,
sa2_name VARCHAR(255), 
total_businesses INT)
""")


#this drops all the columns we do not require
businesses_df = data_businesses.drop(['industry_code', '0_to_50k_businesses', '50k_to_200k_businesses', '200k_to_2m_businesses', '2m_to_5m_businesses', '5m_to_10m_businesses', '10m_or_more_businesses'], axis = 1)

businesses_df.to_sql("businesses", con = conn, if_exists = 'append', index = False)
print("transfer successful")

transfer successful


POPULATION

In [12]:
#POPULATION

conn.execute('DROP TABLE IF EXISTS POPULATION')

#LOAD DATA
import pandas as pd
data_population = pd.read_csv('Population.csv')
population_df = pd.DataFrame(data = data_population)


#this is to limit to SA_2 regions with only population above and equal to 100
population_df = population_df[population_df.total_people >= 100]
#this renames columns to simpler names
population_df = population_df.rename(columns={'0-4_people': 'ages0_4', '5-9_people': 'ages5_9', '10-14_people': 'ages10_14', '15-19_people': 'ages15_19'})

conn.execute("""
CREATE TABLE POPULATION ( 
sa2_code INT PRIMARY KEY, 
sa2_name CHAR(50), 
ages0_4 INT,
ages5_9 INT,
ages10_14 INT, 
ages15_19 INT,
total_people INT)
""")

#this drops all the columns we do not require
population_df = population_df.drop(['20-24_people', '25-29_people', '30-34_people', '35-39_people', '40-44_people', '45-49_people', '50-54_people', '55-59_people', '60-64_people', '65-69_people', '70-74_people', '75-79_people', '80-84_people', '85-and-over_people'], axis = 1)

#brings the dataframe to sql into the table POPULATION of the schema we have already created previously
population_df.to_sql("population", con = conn, if_exists = 'append', index = False)
print("transfer successful")

transfer successful


INCOME

In [30]:
#INCOME

conn.execute('DROP TABLE IF EXISTS INCOME')

#LOAD DATA
import pandas as pd
data_income = pd.read_csv('Income.csv')
income_df = pd.DataFrame(data = data_income)
income_df = income_df[income_df.earners.apply(lambda x: x.isnumeric())]
income_df = income_df[income_df.median_age.apply(lambda x: x.isnumeric())]
income_df = income_df[income_df.median_income.apply(lambda x: x.isnumeric())]
income_df = income_df[income_df.mean_income.apply(lambda x: x.isnumeric())]


conn.execute("""
CREATE TABLE INCOME ( 
sa2_code VARCHAR,
sa2_name CHAR (50), 
earners INT, 
median_age INT, 
median_income int, 
mean_income INT)
""")


#brings the dataframe to sql into the table INCOME of the schema we have already created previously
income_df.to_sql("income", con = conn, if_exists = 'append', index = False)


570

POLLING

In [66]:
#POLLING
import pandas as pd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

conn.execute('DROP TABLE IF EXISTS POLLING')

#LOAD DATA
import pandas as pd
data_polling = pd.read_csv('PollingPlaces2019.csv')
polling_df = pd.DataFrame(data = data_polling)

#this renames columns to simpler names
polling_df = polling_df.rename(columns={'polling_place_id': 'polling_id', 'polling_place_type_id': 'polling_type_id'})

#this is to limit it to Greater Sydney (which has post codes 1000-1920, 2000-2239, 2555-2574, 2740-2786)
polling_df = polling_df[(polling_df.premises_post_code <= 1920) & (polling_df.premises_post_code >= 1000) | (polling_df.premises_post_code <= 2249) & (polling_df.premises_post_code >= 2000)  | (polling_df.premises_post_code <= 2574) & (polling_df.premises_post_code >= 2555) | (polling_df.premises_post_code <= 2786) & (polling_df.premises_post_code >= 2740)]

conn.execute("""CREATE TABLE POLLING ( 
division_id INT, 
division_name CHAR (25),
polling_id INT PRIMARY KEY, 
polling_type_id INT, 
polling_place_name CHAR(255),
premises_name CHAR(255),
premises_suburb CHAR(255),
premises_post_code INT,
latitude FLOAT, 
longitude FLOAT,
geom GEOMETRY(POINT,4326))
""")
#this drops all the columns we do not require
polling_df = polling_df.drop(['FID', 'state', 'premises_address_1', 'premises_address_2', 'premises_address_3', 'premises_state_abbreviation'], axis = 1)

#removes 0 and null values in latitude 
polling_df = polling_df[polling_df.latitude != 0]
polling_df = polling_df[polling_df.latitude.notnull()]

#removes 0 and null values in longitude
polling_df = polling_df[polling_df.longitude != 0]
polling_df = polling_df[polling_df.longitude.notnull()]


#the point data in the original data set is first latitude then longitude and we need it in vice versa

polling_df['geom'] = gpd.points_from_xy(polling_df.longitude, polling_df.latitude)  # creating the geometry column
polling_df = polling_df.drop(columns=['the_geom'])  # removing the old the_geom field

srid = 4326
polling_df['geom'] = polling_df['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

#brings the dataframe to sql into the table POLLING of the schema we have already created previously
polling_df.to_sql("polling", con = conn, if_exists = 'append', index = False,  dtype={'geom': Geometry('POINT', srid)})


404

STOPS

In [87]:
conn.execute('DROP TABLE IF EXISTS STOPS')

#LOAD DATA
import pandas as pd

#creating the table in postgresql pgadmin
conn.execute("""
CREATE TABLE STOPS ( 
stop_id VARCHAR(255) PRIMARY KEY,
stop_code VARCHAR(255), 
stop_name VARCHAR(255), 
location_type VARCHAR(255),
parent_station VARCHAR(255), 
wheelchair_boarding INT,
platform_code VARCHAR,
geom GEOMETRY(POINT,4326))
""")

#imports
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

#converting to geometry point data
stops = pd.read_csv('Stops.csv')
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)  # creating the geometry column
stops = stops.drop(columns=['stop_lat', 'stop_lon'])  # removing the old latitude/longitude fields

#Ensuring the spatial data types from GeoPandas are the same as those expected by PostGIS requires conversion to the Well-Known Text (WKT) format,
#as an intermediate step. We'll also be sure to specify the Spatial Reference Identifier (SRID) - in this case 4326, to represent the WGS84 world
#geodetic coordinate system used by the data set.

#srid = 4326
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))


#loading it in a sql table named stops in pgadmin
stops.to_sql('stops', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})


718

SA2_REGIONS

In [78]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

conn.execute('DROP TABLE IF EXISTS datasa')
conn.execute("""CREATE TABLE datasa ( 
sa2_code21 INT PRIMARY KEY, 
sa2_name21 VARCHAR(255), 
areasqkm21 INT,
geom GEOMETRY(MultiPolygon, 4326))
""")

data_sa2 = gpd.read_file("SA2_2021_AUST_GDA2020.shp")
data_sa2_df = gpd.GeoDataFrame(data = data_sa2)
#data_sa2_df.dropna(subset=['geometry'])

data_sa2_df = data_sa2_df[data_sa2_df.geometry != 0]
data_sa2_df = data_sa2_df[data_sa2_df.geometry.notnull()]

data_sa2_df = data_sa2_df.rename(columns={'SA2_CODE21': 'sa2_code21', 'SA2_NAME21': 'sa2_name21', 'GCC_CODE21': 'gcc_code21', 'GCC_NAME21': 'gcc_name21','STE_CODE21' :'ste_code21', 'STE_NAME21' :'ste_name21', 'AUS_CODE21': 'aus_code21', 'AUS_NAME21' : 'aus_name21', 'AREASQKM21' : 'areasqkm21'})


def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)
srid = 4326
data_sa2_df_copy = data_sa2_df.copy()  # creating a copy of the original for later
data_sa2_df['geom'] = data_sa2_df['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
data_sa2_df = data_sa2_df.drop(columns="geometry")  # deleting the old copy

data_sa2_df = data_sa2_df.drop(columns = ['CHG_FLAG21', 'CHG_LBL21', 'SA3_CODE21', 'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'LOCI_URI21'])
#print(data_sa2_df)

data_sa2_df = data_sa2_df[data_sa2_df.gcc_code21 == "1GSYD"]

data_sa2_df = data_sa2_df.drop(columns = ['gcc_code21', 'gcc_name21', 'ste_code21', 'ste_name21', 'aus_code21', 'aus_name21'])

srid = 4326
data_sa2_df.to_sql("datasa", conn, if_exists='append', index=False, dtype={'geom': Geometry('MultiPolygon', srid)})

373

CATCHMENTS (PRIMARY)

In [22]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import fiona

conn.execute('DROP TABLE IF EXISTS catchmentsprimary')
conn.execute("""CREATE TABLE catchmentsprimary ( 
id TEXT,
geom GEOMETRY(MultiPolygon, 4326))
""")
#data_catchprimary=gpd.read_file("catchments/catchments_primary.shp")
with fiona.open("catchments\catchments_primary.shp") as shapefile:
    data_catch_df = gpd.GeoDataFrame(data = shapefile)
#data_catch_df = data_catch_df[data_catch_df.geometry != 0]
#data_catch_df = data_catch_df[data_catch_df.geometry.notnull()]
data_catch_df = data_catch_df.rename(columns={'USE_ID': 'use_id', 'CATCH_TYPE': 'catch_type', 'USE_DESC': 'use_desc', 'ADD_DATE': 'add_date','KINDERGART' :'kindergart', 'YEAR1' :'year1', 'YEAR2' :'year2', 'YEAR3' :'year3', 'YEAR4' :'year4', 'YEAR5' :'year5', 'YEAR6' :'year6', 'YEAR7' :'year7', 'YEAR8' :'year8', 'YEAR9' :'year9', 'YEAR10' :'year10', 'YEAR11' :'year11', 'YEAR12' :'year12', 'PRIORITY' :'priority'})
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)
srid = 4326
data_catch_df_copy = data_catch_df.copy()  # creating a copy of the original for later
data_catch_df['geom'] = data_catch_df['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
data_catch_df = data_catch_df.drop(columns="geometry")  # deleting the old copy
data_catch_df = data_catch_df.drop(columns="properties")  

srid = 4326
#print(data_catch_df)
data_catch_df.to_sql("catchmentsprimary", conn, if_exists='append', index=False, dtype={'geom': Geometry('MultiPolygon', srid)})


1
2
3
4
5


In [ ]:
CATCHMENTS (SECONDARY)

In [23]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
import fiona

conn.execute('DROP TABLE IF EXISTS catchmentssecondary')
conn.execute("""CREATE TABLE catchmentssecondary ( 
id TEXT,
geom GEOMETRY(MultiPolygon, 4326))
""")
#data_catchprimary=gpd.read_file("catchments/catchments_primary.shp")
with fiona.open("catchments\catchments_secondary.shp") as shapefilesec:
    data_catchsec_df = gpd.GeoDataFrame(data = shapefilesec)
#data_catch_df = data_catch_df[data_catch_df.geometry != 0]
#data_catch_df = data_catch_df[data_catch_df.geometry.notnull()]
data_catchsec_df = data_catchsec_df.rename(columns={'USE_ID': 'use_id', 'CATCH_TYPE': 'catch_type', 'USE_DESC': 'use_desc', 'ADD_DATE': 'add_date','KINDERGART' :'kindergart', 'YEAR1' :'year1', 'YEAR2' :'year2', 'YEAR3' :'year3', 'YEAR4' :'year4', 'YEAR5' :'year5', 'YEAR6' :'year6', 'YEAR7' :'year7', 'YEAR8' :'year8', 'YEAR9' :'year9', 'YEAR10' :'year10', 'YEAR11' :'year11', 'YEAR12' :'year12', 'PRIORITY' :'priority'})
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)
srid = 4326
data_catchsec_df_copy = data_catchsec_df.copy()  # creating a copy of the original for later
data_catchsec_df['geom'] = data_catchsec_df['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
data_catchsec_df = data_catchsec_df.drop(columns="geometry")  # deleting the old copy
data_catchsec_df = data_catchsec_df.drop(columns="properties")  

srid = 4326
#print(data_catch_df)
data_catchsec_df.to_sql("catchmentssecondary", conn, if_exists='append', index=False, dtype={'geom': Geometry('MultiPolygon', srid)})

436

CREATING INDICES IN POPULATION, INCOME, POLLING, DATASA AND STOPS TABLES

In [21]:
#Creating Index on Name of SA2 region in Population table
conn.execute("""
CREATE INDEX NAMECATALOG ON population(sa2_name)
""")

In [22]:
#Creating Index on Name of SA2 region in Income table
conn.execute("""
CREATE INDEX incomeregioncatalog ON income(sa2_name)
""")

In [123]:
#Creating Index on DIVISION Name of SA2 region in Polling table
conn.execute("""
CREATE INDEX divisioncatalog ON polling(division_name)
""")

In [80]:
conn.execute("""
CREATE INDEX sa2_mpg ON datasa (sa2_code21)
""")

SPATIAL INDEX ON STOPS.GEOM

In [28]:
conn.execute("""
CREATE INDEX census_idx ON stops USING GIST(geom)
""")

Z-SCORES

In [13]:
#Calculating the Z-score of Retail Businesses per 1000 people for all SA2 regions 

conn.execute("""
CREATE TABLE businessesfinalsscore
sa2_code21 integer NOT NULL PRIMARY KEY,
businesses double precision
 """)


sql4 = """
INSERT INTO businessesfinalscore
SELECT B.sa2_code, ((B.total_businesses*1000.0)/ P.total_people) as RetailCoverage, (((B.total_businesses*1000.0)/ P.total_people) - (
SELECT AVG((B.total_businesses*1000.0)/ P.total_people)
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Retail Trade')) / (SELECT STDDEV((B.total_businesses*1000.0)/ P.total_people)
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Retail Trade') AS Zscore
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Retail Trade'
"""
query(conn, sql4)

,sa2_code,healthcoverage,zscore
0,102011028,5.976096,-0.148509
1,102011029,4.524068,-0.300496
2,102011030,12.005055,0.482556
3,102011031,10.268189,0.300755
4,102011032,8.432493,0.108608
...,...,...,...
355,128021536,4.288745,-0.325128
356,128021538,5.306175,-0.218631
357,128021607,3.912768,-0.364483
358,128021608,3.943432,-0.361273


In [14]:
#Calculating the Z-score of Health services per 1000 people for all SA2 regions 
conn.execute("""
CREATE TABLE healthfinalscore
sa2_code21 integer NOT NULL PRIMARY KEY,
health double precision
 """)

sql4 = """
INSERT INTO healthfinalscore
SELECT B.sa2_code, ((B.total_businesses*1000.0)/ P.total_people) as HealthCoverage, (((B.total_businesses*1000.0)/ P.total_people) - (
SELECT AVG((B.total_businesses*1000.0)/ P.total_people)
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Health Care and Social Assistance')) / (SELECT STDDEV((B.total_businesses*1000.0)/ P.total_people)
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Health Care and Social Assistance') AS Zscore
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Health Care and Social Assistance'
"""
query(conn, sql4)

,sa2_code,healthcoverage,zscore
0,102011028,9.827357,0.214313
1,102011029,4.976475,-0.381019
2,102011030,9.056445,0.119702
3,102011031,13.713436,0.691238
4,102011032,14.194697,0.750302
...,...,...,...
355,128021536,4.333890,-0.459882
356,128021538,7.616928,-0.056965
357,128021607,4.430635,-0.448008
358,128021608,2.991569,-0.624620


In [31]:
#CREATING stopsfinalscore table



Z_SCORESTOPS =  ("""
INSERT INTO stopsfinalscore
WITH TOTALSTOPS AS
    (SELECT COUNT(*) AS NUMBEROFSTOPS
    FROM datasa as D, stops as S
    WHERE ST_CONTAINS(D.geom, S.geom)),
    STOPNAMES AS 
    (SELECT D.sa2_code21 AS CODE
    FROM datasa as D, stops as S
    WHERE ST_CONTAINS(D.geom, S.geom)
    GROUP BY D.sa2_code21),
    SUMS AS 
    (SELECT SUM(NUMBEROFSTOPS) AS SUMOFSTOPS
    FROM TOTALSTOPS),
    MEANS AS
    (SELECT (SUMOFSTOPS/372) AS MEAN 
    FROM TOTALSTOPS, SUMS),
    DEVIATION AS
    (SELECT STDDEV(NUMBEROFSTOPS) AS STDSTOPS
    FROM TOTALSTOPS)

SELECT D.sa2_code21, D.sa2_name21, (COUNT(*) - 149)/86.346 AS ZSCORESTOPS
FROM datasa as D, stops as S, MEANS, DEVIATION
WHERE ST_CONTAINS(D.geom, S.geom)
GROUP BY D.sa2_code21
""")

query(conn, Z_SCORESTOPS)

,sa2_code21,sa2_name21,zscorestops
0,102011028,Avoca Beach - Copacabana,-0.254789
1,102011029,Box Head - MacMasters Beach,0.764367
2,102011030,Calga - Kulnura,0.266370
3,102011031,Erina - Green Point,1.575058
4,102011032,Gosford - Springfield,2.385750
...,...,...,...
367,128021537,Royal National Park,-1.389757
368,128021538,Sutherland - Kirrawee,0.532740
369,128021607,Engadine,0.833854
370,128021608,Loftus - Yarrawarrah,-0.057907


CALCULATING Z SCORE FOR POLLS

In [29]:
#creatingpollingfinalscore table

Z_SCOREPOLLING = ("""
INSERT INTO pollingfinalscore
WITH TOTALPOLLS AS
    (SELECT COUNT(*) AS NUMBEROFPOLLS
    FROM datasa as D, polling as PO
    WHERE ST_CONTAINS(D.geom, Po.geom)),
    POLLNAMES AS 
    (SELECT D.sa2_code21 AS CODE
    FROM datasa as D, polling as PO
    WHERE ST_CONTAINS(D.geom, Po.geom)
    GROUP BY D.sa2_code21),
    POLLSTOTAL AS 
    (SELECT COUNT(*) AS POLLSNUMBER
    FROM datasa as D, polling as PO
    WHERE ST_CONTAINS(D.geom, Po.geom)
    GROUP BY D.sa2_code21),
    SUMS AS 
    (SELECT SUM(NUMBEROFPOLLS) AS SUMOFPOLLS
    FROM TOTALPOLLS),
    MEANS AS
    (SELECT (SUMOFPOLLS/324) AS MEAN 
    FROM TOTALPOLLS, SUMS),
    DEVIATION AS
    (SELECT STDDEV(POLLSNUMBER) AS STDPOLLS
    FROM POLLSTOTAL)

SELECT D.sa2_code21 as SA2CODE, D.sa2_name21 AS SA2NAME, (COUNT(*) - 4.33)/4.2615713900009118 AS ZSCOREPOLLS
FROM datasa as D, polling as PO, MEANS, DEVIATION
WHERE ST_CONTAINS(D.geom, Po.geom)
GROUP BY D.sa2_code21
""")

query(conn, Z_SCOREPOLLING)

,sa2code,sa2name,zscorepolls
0,102011030,Calga - Kulnura,-0.546747
1,115011290,Baulkham Hills - East,1.565150
2,115011291,Baulkham Hills (West) - Bella Vista,0.157219
3,115011294,Glenhaven,-0.781402
4,115011296,West Pennant Hills,-0.312091
...,...,...,...
319,128021536,Oyster Bay - Como - Jannali,0.626529
320,128021538,Sutherland - Kirrawee,0.391874
321,128021607,Engadine,0.626529
322,128021608,Loftus - Yarrawarrah,-0.546747


CALCULATING Z_SCORE FOR PRIMARY CATCHMENT AREAS

In [25]:
ZSCOREPRIMARY = ("""WITH TOTALPRIMARY AS
    (SELECT COUNT(*) AS NUMBEROFPRIMARY
    FROM datasa as D, catchmentsprimary as cp
    WHERE ST_CONTAINS(D.geom, cp.geom)),
    PRIMARYNAMES AS 
    (SELECT D.sa2_code21 AS CODE
     FROM datasa as D, catchmentsprimary as cp
    WHERE ST_CONTAINS(D.geom, cp.geom)
    GROUP BY D.sa2_code21),
    PRIMARYTOTAL AS 
    (SELECT COUNT(*) AS PRIMARYNUMBER
    FROM datasa as D, catchmentsprimary as cp
    WHERE ST_CONTAINS(D.geom, cp.geom)
    GROUP BY D.sa2_code21),
    SUMS AS 
    (SELECT SUM(NUMBEROFPRIMARY) AS SUMOFPRIMARY
    FROM TOTALPRIMARY),
    MEANS AS
    (SELECT (SUMOFPRIMARY/372) AS MEAN 
    FROM TOTALPRIMARY, SUMS),
    DEVIATION AS
    (SELECT STDDEV(PRIMARYNUMBER) AS STDPRIMARY
    FROM PRIMARYTOTAL)

SELECT D.sa2_code21 as SA2CODE, D.sa2_name21 AS SA2NAME, (COUNT(*) - 0.05107526881720430108)/0.45773770821706345828 AS ZSCOREPRIMARY
FROM datasa as D, catchmentsprimary as cp, MEANS, DEVIATION
WHERE ST_CONTAINS(D.geom, CP.geom)
GROUP BY D.sa2_code21
""")
query(conn, ZSCOREPRIMARY)

,sa2code,sa2name,zscoreprimary
0,102011030,Calga - Kulnura,2.073075
1,116011307,Lalor Park - Kings Langley,2.073075
2,116031316,Lethbridge Park - Tregear,2.073075
3,119011658,Panania (South) - Picnic Point,2.073075
4,123011699,Elderslie - Narellan,2.073075
5,123021438,Claymore - Eagle Vale - Raby,4.257733
6,123021443,Minto - St Andrews,2.073075
7,123021706,Macquarie Fields,4.257733
8,124051470,St Clair,2.073075
9,125021477,Ermington - Rydalmere,4.257733


NOTES: HALF OF ALL ZSCORES ARE ABOVE +3.0, GENERALLY CONSIDERED THE BOUNDARY FOR OUTLIERS IN ZSCORE MATHEMATICS
HENCE WE HAVE DECIDED TO OMIT PRIMARY CATCHMENTS FROM THE SIGMOID CALCULATIONS ENTIRELY

CALCULATING Z_SCORE FOR SECONDARY CATCHMENT AREAS

In [28]:
ZSCORESECONDARY = ("""
WITH TOTALSECONDARY AS
    (SELECT COUNT(*) AS NUMBEROFSECONDARY
    FROM datasa as D, catchmentssecondary as cs
    WHERE ST_CONTAINS(D.geom, cs.geom)),
    SECONDARYNAMES AS 
    (SELECT D.sa2_code21 AS CODE
    FROM datasa as D, catchmentssecondary as cs
    WHERE ST_CONTAINS(D.geom, cs.geom)
    GROUP BY D.sa2_code21),
    SECONDARYTOTAL AS 
    (SELECT COUNT(*) AS SECONDARYNUMBER
    FROM datasa as D, catchmentssecondary as cs
    WHERE ST_CONTAINS(D.geom, cs.geom)
    GROUP BY D.sa2_code21),
    SUMS AS 
    (SELECT SUM(NUMBEROFSECONDARY) AS SUMOFSECONDARY
    FROM TOTALSECONDARY),
    MEANS AS
    (SELECT (SUMOFSECONDARY/372) AS MEAN 
    FROM TOTALSECONDARY, SUMS),
    DEVIATION AS
    (SELECT STDDEV(SECONDARYNUMBER) AS STDSECONDARY
    FROM SECONDARYTOTAL)

SELECT D.sa2_code21 AS CODE, COUNT(*) AS SECONDARYNUMBER
FROM datasa as D, catchmentssecondary as cs
WHERE ST_CONTAINS(D.geom, cs.geom)
GROUP BY D.sa2_code21
""")
query(conn, ZSCORESECONDARY)

,code,secondarynumber


CALCULATING SIGMOID OF Z SCORES FROM ALL TABLES

I MADE THE BUSINESSPLUSHEALTH AND STOPSPLUSPOLLS TABLES THEN ADDED THEM TOGETHER

this code adds the retail z scores and health z scores together

In [ ]:
#creates table for retail businesses' grouped by sa2 region and with z score calculated
conn.execute("""
CREATE TABLE businessesfinalscore
sa2_code21 integer NOT NULL PRIMARY KEY,
health double precision""")


#ADDING SA2CODE, RETAIL ZSCORE INTO A NEW TABLE CALLED businessesfinalscore
conn.execute("""
INSERT INTO businessesfinalscore
SELECT B.sa2_code, (((B.total_businesses*1000.0)/ P.total_people) - (
SELECT AVG((B.total_businesses*1000.0)/ P.total_people)
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Retail Trade')) / (SELECT STDDEV((B.total_businesses*1000.0)/ P.total_people)
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Retail Trade') AS Zscore
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Retail Trade
'""")

In [ ]:
conn.execute("""
CREATE TABLE healthfinalscore
sa2_code21 integer NOT NULL PRIMARY KEY,
health double precision DEFAULT 0.0""")

#ADDING SA2CODE, HEALTH ZSCORE INTO A NEW TABLE CALLED healthfinalscore
conn.execute("""
INSERT INTO healthfinalscore
SELECT B.sa2_code, (((B.total_businesses*1000.0)/ P.total_people) - (
SELECT AVG((B.total_businesses*1000.0)/ P.total_people)
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Health Care and Social Assistance')) / (SELECT STDDEV((B.total_businesses*1000.0)/ P.total_people)
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Health Care and Social Assistance') AS Zscore
FROM businesses as B
INNER JOIN population as P ON B.sa2_code=P.sa2_code
WHERE B.industry_name = 'Health Care and Social Assistance
'""")

In [ ]:
#createtable businessplushealth
conn.execute("""
CREATE TABLE businessplushealth
sa2_code21 VARCHAR,
businessplushealth double precision DEFAULT 0.0""")

#INSERT retail and health zscores into one column in table named businessplushealth
conn.execute("""
INSERT INTO businessplushealth
SELECT b.sa2_code21, b.businesses + h.health as finalzscore
FROM businessesfinalscore as b 
FULL JOIN healthfinalscore as h 
ON b.sa2_code21 = h.sa2_code21
""")

this code adds the stops and polls z scores together

In [ ]:
#stopspluspolls

#CREATE TABLE STOPSPLUSPOLLS
conn.execute("""
sa2_name21 VARCHAR,
stops double precision,
polls double precision,
sa2_code21 INT""")


In [ ]:
#stopspluspollsfinal

#CREATE TABLE STOPSPLUSPOLLSFINAL
conn.execute("""
sa2_code21 VARCHAR,
stopspluspolls DOUBLE PRECISION
""")


In [98]:
#SIGMOID AKA FINAL RESULTS MINUS INCOME
final = ("""select b.sa2_code21, b.businessplushealth + s.stopspluspolls as FINALZSCORE, 1/(1+POWER(2.718, - (b.businessplushealth + s.stopspluspolls))) as SIGMOID
FROM businessplushealth as b, stopspluspollsfinal as s
WHERE b.sa2_code21 = s.sa2_code21
ORDER BY SIGMOID DESC
""")
query(conn, final)


,sa2_code21,finalzscore,sigmoid
0,117031644,44.626970,1.000000
1,115021297,7.560762,0.999479
2,117031645,7.024177,0.999110
3,121011684,5.888816,0.997236
4,115011291,5.466160,0.995788
...,...,...,...
355,116021562,-2.547379,0.072621
356,115031299,-2.744122,0.060436
357,127011726,-2.909543,0.051699
358,128021609,-3.002700,0.047318


In [85]:
#SIGMOID PLUS INCOME
income = ("""select b.sa2_code21, 1/(1+POWER(2.718, - (b.businessplushealth + s.stopspluspolls))) as SIGMOID, i.median_income
FROM businessplushealth as b, stopspluspollsfinal as s
JOIN INCOME as i ON s.sa2_code21 = i.sa2_code
WHERE b.sa2_code21 = s.sa2_code21 AND b.sa2_code21 = i.sa2_code
ORDER BY SIGMOID DESC
""")
query(conn, income)

,sa2_code21,sigmoid,median_income
0,115021297,0.999562,55769
1,115011291,0.996282,60028
2,123021437,0.993845,52550
3,117011320,0.993219,68584
4,124031464,0.986466,54266
...,...,...,...
242,124031465,0.070968,60645
243,119021574,0.069165,36274
244,116021562,0.068786,63303
245,115031299,0.056962,47184


In [ ]:
#this creates another table datasmap to work with that is similar to datasa with the sa2code column being varchar instead of integer

conn.execute("""
CREATE TABLE datasamap(
sa2_code21 VARCHAR,
sa2_name21 VARCHAR,
areasqkm21 INT,
geom GEOMETRY)
""")
#this copies values from earlier datasamap
conn.execute("""
INSERT INTO datasamap
SELECT * 
FROM datasa
""")

THIS CODE VISUALIZES THE GRAPH OF SYDNEY WITH SIGMOID VALUES OF
EACH SA2 REGION ALONG WITH MEDIAN INCOME AVAILABLE ON GEOMETRY VIEWER MAP

In [94]:
graphvis = ("""
WITH SIGMA AS
    (SELECT b.sa2_code21 AS SA2CODE, da.sa2_name21 AS SA2NAME, b.businessplushealth + s.stopspluspolls as FINALZSCORE, 1/(1+POWER(2.718, - (b.businessplushealth + s.stopspluspolls))) as SIGMOID, i.median_income AS MEDIANINCOME
    FROM (businessplushealth as b JOIN datasamap as da ON b.sa2_code21 = da.sa2_code21), stopspluspollsfinal as s
    JOIN INCOME as i ON s.sa2_code21 = i.sa2_code
    WHERE b.sa2_code21 = s.sa2_code21 AND b.sa2_code21 = i.sa2_code
    ORDER BY SIGMOID DESC)

    SELECT d.sa2_code21 as SA2_CODE, d.sa2_name21 AS SA2_NAME, s.SIGMOID AS SIGMOID, s.MEDIANINCOME as MEDIANINCOME, d.GEOM as GEOM
    FROM datasamap as d
    JOIN SIGMA AS s ON d.sa2_name21 = s.SA2NAME
""")

query(conn, graphvis)

,sa2_code,sa2_name,sigmoid,medianincome,geom
0,102011028,Avoca Beach - Copacabana,0.451826,52450,0106000020E6100000010000000103000000010000005E...
1,102011029,Box Head - MacMasters Beach,0.521173,48724,0106000020E61000000100000001030000000100000010...
2,102011030,Calga - Kulnura,0.581571,46228,0106000020E61000000200000001030000000100000085...
3,102011031,Erina - Green Point,0.932492,48292,0106000020E61000000100000001030000000100000041...
4,102011032,Gosford - Springfield,0.965064,51999,0106000020E6100000010000000103000000010000007E...
...,...,...,...,...,...
242,128021536,Oyster Bay - Como - Jannali,0.746812,64428,0106000020E610000001000000010300000001000000EF...
243,128021538,Sutherland - Kirrawee,0.660118,62453,0106000020E61000000100000001030000000100000089...
244,128021607,Engadine,0.659860,60914,0106000020E6100000010000000103000000010000008E...
245,128021608,Loftus - Yarrawarrah,0.164265,60469,0106000020E610000001000000010300000001000000A1...


TASK 3

Importing Housing table, creating schema and adding to postgreSQL 

In [39]:
#HOUSING

conn.execute('DROP TABLE IF EXISTS housing')

#LOAD DATA
import pandas as pd
data_housing = pd.read_csv('housingnum.csv')
housing_df = pd.DataFrame(data = data_housing)

#creating the table in postgresql pgadmin
conn.execute("""
CREATE TABLE housing ( 
sa2_code INT,
areaname VARCHAR(255),
newhouses INT, 
totaldwellings INT, 
valueofnew INT,
valueoftotal INT
)
""")

housing_df = housing_df.drop(['New other residential building', 'Value of new other residential building', 'Value of alterations & additions including conversions to residential building', 'Value of total residential building', 'Value of non-residential building'], axis = 1)

#brings the dataframe to sql into the table housing of the schema we have already created previously
housing_df.to_sql("housing", con = conn, if_exists = 'append', index = False)

transfer successful


Importing CHILDCARE table, creating schema and adding to postgreSQL

In [55]:
#CHILDCARE WHICH CONTAINS SPATIAL DATA

conn.execute('DROP TABLE IF EXISTS childcare')

#LOAD DATA
import pandas as pd
import geopandas as gpd
data_childcare = pd.read_csv('childcare.csv')

#creating the table in postgresql pgadmin
conn.execute("""
CREATE TABLE childcare ( 
location_id VARCHAR PRIMARY KEY,
geom GEOMETRY(Point,4326)
)
""")

data_childcare= data_childcare.drop(columns=['geom'])

#Ensuring the spatial data types from GeoPandas are the same as those expected by PostGIS requires conversion to the Well-Known Text (WKT) format,
#as an intermediate step. We'll also be sure to specify the Spatial Reference Identifier (SRID) - in this case 4326, to represent the WGS84 world
#geodetic coordinate system used by the data set.

data_childcare['geom'] = gpd.points_from_xy(data_childcare.longitude, data_childcare.latitude)  # creating the geometry column

#srid = 4326
data_childcare['geom'] = data_childcare['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

#drop all these columns as we don't need them
data_childcare = data_childcare.drop(['id', 'typeofprovider', 'provider_name', 'address', 'suburb', 'state', 'postcode', 'latitude', 'longitude'], axis = 1)

#brings the dataframe to sql into the table housing of the schema we have already created previously
data_childcare.to_sql("childcare", con = conn, if_exists = 'append', index = False, dtype={'geom': Geometry('POINT', srid)})

476

Importing disability table, creating schema and adding to postgreSQL

In [80]:
#DISABILITY 

conn.execute('DROP TABLE IF EXISTS disability')

#LOAD DATA
import pandas as pd
data_disability = pd.read_csv('disability.csv')
disability_df = pd.DataFrame(data = data_disability)

disability_df['per1000'] = disability_df.apply(lambda row: (row['Disability']*-1000/row['Population']),
                                   axis=1)


#creating the table in postgresql pgadmin
conn.execute("""
CREATE TABLE disability ( 
sa2_code INT PRIMARY KEY,
sa2_name VARCHAR,
per1000 FLOAT
)
""")

#drop all these columns as we don't need them
disability_df = disability_df.drop(['Disability', 'Relative root mean square error', 'Proportion of population', 'Unnamed: 4', 'xyz', '95% Confidence Interval of proportion', 'Population'], axis = 1)
#print(disability_df)
#brings the dataframe to sql into the table housing of the schema we have already created previously
disability_df.to_sql("disability", con = conn, if_exists = 'append', index = False)

168

calculating z score for Housing table

In [83]:
housingzscore = ("""
WITH TOTALROWS AS
    (SELECT COUNT(*) AS ROWSTOTAL
    FROM housing WHERE totaldwellings is not NULL),
    SUMOFHOUSES AS 
    (SELECT sum(totaldwellings) as SUMS
    FROM housing),
    MEANTOTAL AS 
    (SELECT SUMS/ROWSTOTAL AS MEAN
    FROM TOTALROWS, SUMOFHOUSES),
    DEVIATION AS
    (SELECT STDDEV(TOTALDWELLINGS) AS STD
    FROM housing)

SELECT h.sa2_code, (h.totaldwellings - m.MEAN)/d.STD AS ZSCORES
FROM housing as h, MEANTOTAL AS m, DEVIATION AS d
WHERE h.sa2_code is not NULL
""")

query(conn, housingzscore)

,sa2_code,zscores
0,102011028,-0.323884
1,102011029,-0.322363
2,102011030,-0.361899
3,102011031,-0.364940
4,102011032,-0.036494
...,...,...
428,128021537,-0.383187
429,128021538,-0.191593
430,128021607,-0.220484
431,128021608,-0.364940


calculating z score for childcare table

In [82]:
childcaresqlzscore = ("""
WITH TOTALCHILD AS
    (SELECT COUNT(*) AS NUMBEROFCHILD
    FROM datasa as D, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)),
    CHILDNAMES AS 
    (SELECT D.sa2_code21 AS CODE, COUNT(*) AS CHILDNUMBER
    FROM datasa as D, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)
    GROUP BY D.sa2_code21),
    SUMS AS 
    (SELECT SUM(NUMBEROFCHILD) AS SUMOFCHILD
    FROM TOTALCHILD),
    MEANS AS
    (SELECT (SUMOFCHILD/350) AS MEAN 
    FROM TOTALCHILD, SUMS),
    DEVIATION AS
    (SELECT STDDEV(CHILDNUMBER) AS DEVCHILD
    FROM CHILDNAMES
    WHERE CHILDNUMBER IS NOT NULL)

SELECT D.sa2_code21, D.sa2_name21, (COUNT(*) - 6.3428571428571429)/3.6607016812378472 AS ZSCORESTOPS
FROM datasa as D, MEANS as m, MEANS, DEVIATION as de, childcare as cc
WHERE ST_CONTAINS(D.geom, cc.geom)
GROUP BY D.sa2_code21
""")
                      
query(conn, childcaresqlzscore)

,sa2_code21,sa2_name21,zscorestops
0,102011028,Avoca Beach - Copacabana,-1.459517
1,102011029,Box Head - MacMasters Beach,-0.913174
2,102011030,Calga - Kulnura,-1.186346
3,102011031,Erina - Green Point,1.272200
4,102011032,Gosford - Springfield,0.725856
...,...,...,...
345,128021536,Oyster Bay - Como - Jannali,0.999028
346,128021538,Sutherland - Kirrawee,1.545371
347,128021607,Engadine,1.818543
348,128021608,Loftus - Yarrawarrah,-1.459517


calculating z score for disability table 

In [84]:
disabilityzscore = ("""
WITH TOTALDIS AS
    (SELECT COUNT(*) AS ROWSTOTAL
    FROM disability WHERE per1000 is not NULL),
    SUMOFDIS AS 
    (SELECT sum(per1000) as SUMS
    FROM disability),
    MEANTOTAL AS 
    (SELECT SUMS/ROWSTOTAL AS MEAN
    FROM TOTALDIS, SUMOFDIS),
    DEVIATION AS
    (SELECT STDDEV(per1000) AS STD
    FROM disability)

SELECT dis.sa2_code, (dis.per1000 - m.MEAN)/d.STD AS ZSCORES
FROM disability as dis, MEANTOTAL AS m, DEVIATION AS d
WHERE dis.sa2_code is not NULL
""")

query(conn, disabilityzscore)

,sa2_code,zscores
0,101021007,-0.449778
1,101021008,-0.449711
2,101021009,0.460537
3,101021010,0.545185
4,101021011,0.738524
...,...,...
2163,801101135,0.968202
2164,801101138,2.540139
2165,801101139,1.058736
2166,801111140,0.225750


SIGMOID VALUES OF ONLY STUDENTS' DATA SOURCES

In [92]:
#to calculate sigmoid of only our three datasources

ourdata = ("""WITH TOTALDIS AS
    (SELECT COUNT(*) AS ROWSTOTAL
    FROM disability WHERE per1000 is not NULL),
    SUMOFDIS AS 
    (SELECT sum(per1000) as SUMS
    FROM disability),
    MEANTOTAL AS 
    (SELECT SUMS/ROWSTOTAL AS MEAN
    FROM TOTALDIS, SUMOFDIS),
    DEVIATION AS
    (SELECT STDDEV(per1000) AS STD
    FROM disability),
    DIS_ZSCORE AS 
    (SELECT dis.sa2_code AS SA2CODE, (dis.per1000 - m.MEAN)/d.STD AS ZSCORES
    FROM disability as dis, MEANTOTAL AS m, DEVIATION AS d
    WHERE dis.sa2_code is not NULL),
    TOTALCHILD AS
    (SELECT COUNT(*) AS NUMBEROFCHILD
    FROM datasa as D, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)),
    CHILDNAMES AS 
    (SELECT D.sa2_code21 AS CODE, COUNT(*) AS CHILDNUMBER
    FROM datasa as D, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)
    GROUP BY D.sa2_code21),
    SUMS AS 
    (SELECT SUM(NUMBEROFCHILD) AS SUMOFCHILD
    FROM TOTALCHILD),
    MEANS AS
    (SELECT (SUMOFCHILD/350) AS MEAN 
    FROM TOTALCHILD, SUMS),
    CHILD_STDEV AS
    (SELECT STDDEV(CHILDNUMBER) AS DEVCHILD
    FROM CHILDNAMES
    WHERE CHILDNUMBER IS NOT NULL),
    CHILD_ZSCORE AS
    (SELECT D.sa2_code21 AS SA2CODE, D.sa2_name21 AS SA2NAME, (COUNT(*) - 6.3428571428571429)/3.6607016812378472 AS ZSCORESTOPS
    FROM datasa as D, MEANS as m, MEANS, CHILD_STDEV as cds, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)
    GROUP BY D.sa2_code21),
    TOTALROWS AS
    (SELECT COUNT(*) AS ROWSTOTAL
    FROM housing WHERE totaldwellings is not NULL),
    SUMOFHOUSES AS 
    (SELECT sum(totaldwellings) as SUMS
    FROM housing),
    HOUSEMEANS AS 
    (SELECT SUMS/ROWSTOTAL AS MEAN
    FROM TOTALROWS, SUMOFHOUSES),
    HOMEDEVIATION AS
    (SELECT STDDEV(TOTALDWELLINGS) AS STD
    FROM housing),
    HOUSE_ZSCORE AS 
    (SELECT h.sa2_code AS SA2CODE, (h.totaldwellings - m.MEAN)/d.STD AS ZSCORES
    FROM housing as h, HOUSEMEANS AS m, HOMEDEVIATION AS d
    WHERE h.sa2_code is not NULL)
    
SELECT dz.SA2CODE AS SA2CODE, ch.SA2NAME AS SA2NAME, 1/(1 + POWER(2.718, - (hz.ZSCORES + ch.ZSCORESTOPS + dz.ZSCORES))) AS SIGMOID
FROM DIS_ZSCORE as dz, HOUSE_ZSCORE as hz, CHILD_ZSCORE as ch
WHERE dz.SA2CODE = hz.SA2CODE AND dz.SA2CODE = ch.SA2CODE
ORDER BY SIGMOID DESC
""")

query(conn, ourdata)

,sa2code,sa2name,sigmoid
0,122031429,Freshwater - Brookvale,0.981203
1,125041493,Toongabbie - Constitution Hill,0.980936
2,116021563,Quakers Hill,0.977394
3,118011341,Bondi Junction - Waverly,0.967438
4,125021477,Ermington - Rydalmere,0.935658
...,...,...,...
240,124011455,Wentworth Falls,0.067519
241,102021046,Budgewoi - Buff Point - Halekulani,0.054876
242,115031299,Bilpin - Colo - St Albans,0.051657
243,102021052,Summerland Point - Gwandalan,0.049910


In [97]:
FINALSQL = ("""
WITH TOTALDIS AS
    (SELECT COUNT(*) AS ROWSTOTAL
    FROM disability WHERE per1000 is not NULL),
    SUMOFDIS AS 
    (SELECT sum(per1000) as SUMS
    FROM disability),
    MEANTOTAL AS 
    (SELECT SUMS/ROWSTOTAL AS MEAN
    FROM TOTALDIS, SUMOFDIS),
    DEVIATION AS
    (SELECT STDDEV(per1000) AS STD
    FROM disability),
    DIS_ZSCORE AS 
    (SELECT dis.sa2_code AS SA2CODE, (dis.per1000 - m.MEAN)/d.STD AS ZSCORES
    FROM disability as dis, MEANTOTAL AS m, DEVIATION AS d
    WHERE dis.sa2_code is not NULL),
    TOTALCHILD AS
    (SELECT COUNT(*) AS NUMBEROFCHILD
    FROM datasa as D, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)),
    CHILDNAMES AS 
    (SELECT D.sa2_code21 AS CODE, COUNT(*) AS CHILDNUMBER
    FROM datasa as D, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)
    GROUP BY D.sa2_code21),
    SUMS AS 
    (SELECT SUM(NUMBEROFCHILD) AS SUMOFCHILD
    FROM TOTALCHILD),
    MEANS AS
    (SELECT (SUMOFCHILD/350) AS MEAN 
    FROM TOTALCHILD, SUMS),
    CHILD_STDEV AS
    (SELECT STDDEV(CHILDNUMBER) AS DEVCHILD
    FROM CHILDNAMES
    WHERE CHILDNUMBER IS NOT NULL),
    CHILD_ZSCORE AS
    (SELECT D.sa2_code21 AS SA2CODE, D.sa2_name21 AS SA2NAME, (COUNT(*) - 6.3428571428571429)/3.6607016812378472 AS ZSCORESTOPS
    FROM datasa as D, MEANS as m, MEANS, CHILD_STDEV as cds, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)
    GROUP BY D.sa2_code21),
    TOTALROWS AS
    (SELECT COUNT(*) AS ROWSTOTAL
    FROM housing WHERE totaldwellings is not NULL),
    SUMOFHOUSES AS 
    (SELECT sum(totaldwellings) as SUMS
    FROM housing),
    HOUSEMEANS AS 
    (SELECT SUMS/ROWSTOTAL AS MEAN
    FROM TOTALROWS, SUMOFHOUSES),
    HOMEDEVIATION AS
    (SELECT STDDEV(TOTALDWELLINGS) AS STD
    FROM housing),
    HOUSE_ZSCORE AS 
    (SELECT h.sa2_code AS SA2CODE, (h.totaldwellings - m.MEAN)/d.STD AS ZSCORES
    FROM housing as h, HOUSEMEANS AS m, HOMEDEVIATION AS d
    WHERE h.sa2_code is not NULL),
    OURDATA AS
    (SELECT dz.SA2CODE AS SA2CODE, ch.SA2NAME AS SA2NAME, hz.ZSCORES + ch.ZSCORESTOPS + dz.ZSCORES AS OURZSCORE
    FROM DIS_ZSCORE as dz, HOUSE_ZSCORE as hz, CHILD_ZSCORE as ch
    WHERE dz.SA2CODE = hz.SA2CODE AND dz.SA2CODE = ch.SA2CODE),
    ORIGINAL AS 
    (SELECT b.sa2_code21 AS SA2CODE, da.sa2_name21 AS SA2NAME, b.businessplushealth + s.stopspluspolls as OGZSCORE
    FROM (businessplushealth as b JOIN datasamap as da ON b.sa2_code21 = da.sa2_code21), stopspluspollsfinal as s
    WHERE b.sa2_code21 = s.sa2_code21)


SELECT og.SA2CODE as SA2CODE, og.SA2NAME AS SA2NAME, (1/(1 + POWER(2.718, (- (og.OGZSCORE + od.OURZSCORE))))) AS SIGMOID
FROM OURDATA AS od, ORIGINAL as og
WHERE og.SA2NAME = od.SA2NAME
ORDER BY SIGMOID DESC

""")

query(conn, FINALSQL)

,sa2code,sa2name,sigmoid
0,115021297,Dural - Kenthurst - Wisemans Ferry,0.999914
1,115011291,Baulkham Hills (West) - Bella Vista,0.999702
2,122031429,Freshwater - Brookvale,0.999296
3,118011341,Bondi Junction - Waverly,0.998972
4,123021437,Campbelltown - Woodbine,0.998857
...,...,...,...
238,128021609,Woronora Heights,0.019755
239,127031601,Warwick Farm,0.018909
240,102021045,Blue Haven - San Remo,0.018142
241,102021052,Summerland Point - Gwandalan,0.006032


to calculate sigmoid of given data sources + ours with median income attached

In [88]:
FINALSQLINCOME = ("""WITH TOTALDIS AS
    (SELECT COUNT(*) AS ROWSTOTAL
    FROM disability WHERE per1000 is not NULL),
    SUMOFDIS AS 
    (SELECT sum(per1000) as SUMS
    FROM disability),
    MEANTOTAL AS 
    (SELECT SUMS/ROWSTOTAL AS MEAN
    FROM TOTALDIS, SUMOFDIS),
    DEVIATION AS
    (SELECT STDDEV(per1000) AS STD
    FROM disability),
    DIS_ZSCORE AS 
    (SELECT dis.sa2_code AS SA2CODE, (dis.per1000 - m.MEAN)/d.STD AS ZSCORES
    FROM disability as dis, MEANTOTAL AS m, DEVIATION AS d
    WHERE dis.sa2_code is not NULL),
    TOTALCHILD AS
    (SELECT COUNT(*) AS NUMBEROFCHILD
    FROM datasa as D, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)),
    CHILDNAMES AS 
    (SELECT D.sa2_code21 AS CODE, COUNT(*) AS CHILDNUMBER
    FROM datasa as D, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)
    GROUP BY D.sa2_code21),
    SUMS AS 
    (SELECT SUM(NUMBEROFCHILD) AS SUMOFCHILD
    FROM TOTALCHILD),
    MEANS AS
    (SELECT (SUMOFCHILD/350) AS MEAN 
    FROM TOTALCHILD, SUMS),
    CHILD_STDEV AS
    (SELECT STDDEV(CHILDNUMBER) AS DEVCHILD
    FROM CHILDNAMES
    WHERE CHILDNUMBER IS NOT NULL),
    CHILD_ZSCORE AS
    (SELECT D.sa2_code21 AS SA2CODE, D.sa2_name21 AS SA2NAME, (COUNT(*) - 6.3428571428571429)/3.6607016812378472 AS ZSCORESTOPS
    FROM datasa as D, MEANS as m, MEANS, CHILD_STDEV as cds, childcare as cc
    WHERE ST_CONTAINS(D.geom, cc.geom)
    GROUP BY D.sa2_code21),
    TOTALROWS AS
    (SELECT COUNT(*) AS ROWSTOTAL
    FROM housing WHERE totaldwellings is not NULL),
    SUMOFHOUSES AS 
    (SELECT sum(totaldwellings) as SUMS
    FROM housing),
    HOUSEMEANS AS 
    (SELECT SUMS/ROWSTOTAL AS MEAN
    FROM TOTALROWS, SUMOFHOUSES),
    HOMEDEVIATION AS
    (SELECT STDDEV(TOTALDWELLINGS) AS STD
    FROM housing),
    HOUSE_ZSCORE AS 
    (SELECT h.sa2_code AS SA2CODE, (h.totaldwellings - m.MEAN)/d.STD AS ZSCORES
    FROM housing as h, HOUSEMEANS AS m, HOMEDEVIATION AS d
    WHERE h.sa2_code is not NULL),
    OURDATA AS
    (SELECT dz.SA2CODE AS SA2CODE, ch.SA2NAME AS SA2NAME, hz.ZSCORES + ch.ZSCORESTOPS + dz.ZSCORES AS OURZSCORE
    FROM DIS_ZSCORE as dz, HOUSE_ZSCORE as hz, CHILD_ZSCORE as ch
    WHERE dz.SA2CODE = hz.SA2CODE AND dz.SA2CODE = ch.SA2CODE),
    ORIGINAL AS 
    (SELECT b.sa2_code21 AS SA2CODE, da.sa2_name21 AS SA2NAME, b.businessplushealth + s.stopspluspolls as OGZSCORE, i.median_income AS MEDIANINCOME
    FROM (businessplushealth as b JOIN datasamap as da ON b.sa2_code21 = da.sa2_code21), stopspluspollsfinal as s
    JOIN INCOME as i ON s.sa2_code21 = i.sa2_code
    WHERE b.sa2_code21 = s.sa2_code21 AND b.sa2_code21 = i.sa2_code)


SELECT og.SA2CODE as SA2CODE, og.SA2NAME AS SA2NAME, (1/(1 + POWER(2.718, (- (og.OGZSCORE + od.OURZSCORE))))) AS SIGMOID
FROM OURDATA AS od, ORIGINAL as og
WHERE og.SA2NAME = od.SA2NAME
ORDER BY SIGMOID DESC
""")

query(conn, FINALSQLINCOME)


,sa2code,sa2name,sigmoid
0,115021297,Dural - Kenthurst - Wisemans Ferry,0.999914
1,115011291,Baulkham Hills (West) - Bella Vista,0.999702
2,122031429,Freshwater - Brookvale,0.999296
3,118011341,Bondi Junction - Waverly,0.998972
4,123021437,Campbelltown - Woodbine,0.998857
...,...,...,...
238,128021609,Woronora Heights,0.019755
239,127031601,Warwick Farm,0.018909
240,102021045,Blue Haven - San Remo,0.018142
241,102021052,Summerland Point - Gwandalan,0.006032
